In [4]:
import pandas as pd
import numpy as np



va_activity_loc_assign = pd.read_csv('va_activity_location_assignment.csv.gz', compression='gzip').iloc[:,1:]
va_activity_locations = pd.read_csv('va_activity_locations.csv.gz', compression='gzip').iloc[:,1:]
va_disease_outcome_target = pd.read_csv('va_disease_outcome_target.csv.gz', compression='gzip').iloc[:,1:]
va_disease_outcome_training = pd.read_csv('va_disease_outcome_training.csv.gz', compression='gzip')
va_household = pd.read_csv('va_household.csv.gz', compression = 'gzip').iloc[:,1:]
va_person = pd.read_csv('va_person.csv.gz', compression='gzip')
va_population_network = pd.read_csv('va_population_network.csv.gz', compression='gzip')
va_residence_locations = pd.read_csv('va_residence_locations.csv.gz', compression='gzip').iloc[:,1:]
# va_residence_locations.head()

In [19]:
##### Locations they been to in past week - and how frequent it is
##### Spread stats for last week at said locations
##### Age, sex
##### Household number
##### Raw duration of time spent with infected neighbors
##### One hot encoding of types of activities participated in or weighted with how long they did those
#####

class GraphStats:
    def __init__(self, va_activity_loc_assign, va_activity_locations, va_disease_outcome_target, va_disease_outcome_training, va_household, va_person, va_population_network, va_residence_locations) -> None:
        self.va_activity_loc_assign = va_activity_loc_assign
        self.va_activity_locations = va_activity_locations
        self.va_disease_outcome_target = va_disease_outcome_target
        self.va_disease_outcome_training = va_disease_outcome_training
        self.va_household = va_household
        self.va_person = va_person
        self.va_population_network = va_population_network
        self.va_residence_locations = va_residence_locations

    ##gets age and sex of a specified person
    def get_age_sex(self, pid):
        res = self.va_person.loc[self.va_person.loc['pid']==pid]
        return (res['age'],res['sex'])

    ### gets household members including person
    def household_members(self, pid):
        hid = self.va_person.query('pid == @pid')['hid'][0]
        return self.va_person.query('hid == @hid')['pid'].to_numpy()
    
    
    def get_raw_time_with_infected_day(self, pid, day):
        interactions = self.va_population_network.query('pid1 == @pid or pid2 == @pid')
        pid1 = interactions['pid1'].to_numpy()
        pid2 = interactions['pid2'].to_numpy()
        time = 0
        pids = np.concatenate((pid1, pid2))
        pids = pids[pids!=pid]
        disease_info = self.va_disease_outcome_training.query('pid in @pids and day == @day').query('state == "I"')
        #print("info: ", disease_info)
        for row in range(disease_info.shape[0]):
            print('row: ', row)
        
        

        return disease_info.shape[0] !=0
    
    
    def get_activity_vector(self, pid):
        user_activities = self.va_activity_loc_assign.query('pid == @pid')
        activity_vector = np.zeros(6)
        for i in range(1, 7):
            activity_vector[i-1] += user_activities.query('activity_type == @i')['duration'].sum()
        return activity_vector

In [20]:
test = GraphStats(va_activity_loc_assign, va_activity_locations, va_disease_outcome_target, va_disease_outcome_training, va_household, va_person, va_population_network, va_residence_locations)  
#print(va_person.head())
#print(va_person.tail())
print(test.get_activity_vector(480694))


[73739.     0.   300.  4560.     0.     0.]


,hid,pid,activity_number,activity_type,start_time,duration,lid
0,213614,480693,19,1,0,14400,1001000351
1,213614,480693,20,1,14400,37200,1001000351
2,213614,480693,22,3,52500,300,5513
3,213614,480693,24,1,54060,960,1001000351
4,213614,480693,26,4,57720,2280,84421
5,213614,480693,28,4,62040,2280,84421
6,213614,480693,30,1,65220,21179,1001000351
7,213614,480694,25,1,0,14400,1001000351
8,213614,480694,26,1,14400,37200,1001000351
9,213614,480694,28,3,52500,300,26385


SyntaxError: expected ':' (782239618.py, line 1)